# Testing explainer 

In [ ]:
# intel-tensorflow==2.9.1
# shap @ git+https://github.com/slundberg/shap@v0.41.0
from explainer.explainers import feature_attributions_explainer, metrics_explainer
feature_attributions_explainer.entry_points

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

url = 'https://www.cs.toronto.edu/~kriz/cifar-10-sample/dog10.png'
image = Image.open(requests.get(url, stream=True).raw)
feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
preds = outputs.logits.argmax(dim=1)

classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]
classes[preds[0]]

In [ ]:
from datasets import list_datasets, load_dataset
from pprint import pprint

datasets_list = list_datasets() 
pprint(datasets_list,compact=True) 

In [ ]:
from IPython.display import Image
Image(filename='images/dog10.png')

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, BasicTokenizer
import torch
import numpy as np
import scipy as sp
import shap


def f(x):
    model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english")
    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128, truncation=True) for v in x])
    attention_mask = (tv!=0).type(torch.int64)
    logit_arr = model(tv,attention_mask=attention_mask).logits.detach().cpu().numpy()
    scores = sp.special.expit(logit_arr)
    return scores

def custom_tokenizer(text, return_offsets_mapping=True):
    text = text.lower()
    wordpunct = BasicTokenizer()
    splitted_text_list = wordpunct.tokenize(text)
    pos_list = []
    pos=0
    for item in splitted_text_list: 
      start = text.find(item, pos)
      end = start + len(item)
      pos_list.append((start, end))
      pos = end
    out={}
    out["input_ids"] = splitted_text_list
    if return_offsets_mapping: 
      out['offset_mapping']=pos_list
    return out

masker = shap.maskers.Text(custom_tokenizer)
masker.mask_token = ''
explainer = shap.Explainer(f, masker)

In [ ]:
shap_values = explainer(X[:100])

# plot the SHAP values for the positive class
shap.plots.beeswarm(shap_values[...,1])

In [ ]:
#%%script false --no-raise-error

from explainer.explainers import metrics_explainer

pstats = metrics_explainer['pstats']('explainer = shap.Explainer(f, masker)')
pstats.summary

In [ ]:
pstats.report

In [ ]:
%pinfo explainer

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram

sample_text = 'Crossover comfort food with a redemptive twist'
Z=masker.clustering(sample_text)
fig = plt.figure(figsize=(8, 4))
dn = dendrogram(Z)
plt.show()

In [ ]:
from transformers import BartForSequenceClassification, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')

# pose sequence as a NLI premise and label (politics) as a hypothesis
premise = 'Counterfactuals'
hypothesis = 'Machine Learning'

# run through model pre-trained on MNLI
input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
logits = model(input_ids)[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
true_prob = probs[:,1].item() * 100
print(f'Probability that the label is true: {true_prob:0.2f}%')

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')

sentence = 'What explainability methods are most often used with the RoBERTa model?'
labels = ['Counterfactual', 'Contrastive', 'GradCAM', 'ScoreCAM',
          'LayerCAM', 'Smooth gradient', 'Integrated gradient',
          'Partial Dependence Plot', 'Accumulated Local Effects',
          'TCAV'
         ]

# run inputs through model and mean-pool over the sequence
# dimension to get sequence-level representations
inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                     return_tensors='pt',
                                     pad_to_max_length=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
output = model(input_ids, attention_mask=attention_mask)[0]
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

# now find the labels with the highest cosine similarities to
# the sentence
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
for ind in closest:
    print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')